In [1]:
import numpy as np
import random
import pandas as pd

In [2]:
import mosek

In [6]:
def msk_relent(task, t, x, y):
    v = msk_newvar(task, 1)
    c = msk_newcon(task, 1)
    task.putaij(c, v, 1.0)
    task.putaij(c, t, 1.0)

    task.putconbound(c, mosek.boundkey.fx, 0.0, 0.0)
    task.appendcone(mosek.conetype.pexp, 0.0, [y, x, v])

def msk_newvar(task, num):  # free
    v = task.getnumvar()
    task.appendvars(num)
    for i in range(num):
        task.putvarbound(v+i, mosek.boundkey.fr, -inf, inf)
    return v

def msk_newvar_fx(task, num, val):  # fixed
    v = task.getnumvar()
    task.appendvars(num)
    for i in range(num):
        task.putvarbound(v+i, mosek.boundkey.fx, val, val)
    return v

def msk_newcon(task, num):
    c = task.getnumcon()
    task.appendcons(num)
    return c

def msk_sq(task, t, x): # t >= x^2
    task.appendcone(mosek.conetype.rquad, 0.0, [msk_newvar_fx(task, 1, 0.5), t, x])

In [11]:
ref1=[0.5,0.5]
ref2=[0.3,0.7]
T=len(ref1)
inf=100
env  = mosek.Env()
task = env.Task(0,0)

# variables -------------------------------------------------------------
u = msk_newvar(task, T)
u2 = msk_newvar(task, T)
eta1 = msk_newvar(task, T) # eta1 is the relative entropy of u over ref1
eta2 = msk_newvar(task, T)

# constraints ------------------------------------------------------------
# 0<u<1 
for i in range(T):
    task.putvarbound(u+i, mosek.boundkey.ra, 0, 1)
    task.putvarbound(eta1+i , mosek.boundkey.ra, -inf, inf)
    task.putvarbound(eta2+i , mosek.boundkey.ra, -inf, inf)

# sum(u)=1
c = msk_newcon(task, 1)
task.putarow(c, range(u,u+T),[1.0]*T)
task.putconbound(c, mosek.boundkey.fx, 1.0, 1.0)  

# u=u2
for i in range(T):
    c = msk_newcon(task, 1)
    task.putarow(c, [u+i,u2+i],[1.0,-1.0])
    task.putconbound(c, mosek.boundkey.fx, 0.0, 0.0)  

# max relative entropy
for i in range(T):
    msk_relent(task, eta1+i, u+i, msk_newvar_fx(task, 1, ref1[i])) #(t,x,y) t >= x * log(x/y), x,y>=0  --> -t <= -x * log x 
    msk_relent(task, eta2+i, u2+i, msk_newvar_fx(task, 1, ref2[i])) #(t,x,y)

# objective ----------------------------------------------------------------
task.putclist(range(eta1,eta2+T),[1]*2*T)
#task.putclist(range(task.getnumvar()),[1.0]*task.getnumvar())

# Input the objective sense (minimize/maximize)
task.putobjsense(mosek.objsense.minimize)

# get solution
task.optimize()
xx = [0.0]*task.getnumvar()
task.getxx(mosek.soltype.itr,xx)

In [12]:
xx

[0.3956873424021416,
 0.6043126579891378,
 0.3956873423168808,
 0.6043126589136365,
 -0.09258436505809503,
 0.11450734590810788,
 0.10954288378043245,
 -0.08882708424681325,
 0.5,
 0.09258436505809503,
 0.3,
 -0.10954288378043245,
 0.5,
 -0.11450734590810788,
 0.7,
 0.08882708424681325]